# EDS 296 Final Project
Authors: Haylee Oyler, Emma Bea Mitchell, Kimberlee Wong

## Region of Interest: Southeast Asia

In [1]:
# Import packages
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import intake

In [2]:
# Open the CMIP6 data catalog, store as a variable
catalog = intake.open_esm_datastore('https://cmip6-pds.s3.amazonaws.com/pangeo-cmip6.json')

# Convert the catalog to a df for easier access
cat_df = catalog.df

In [3]:
# Specify search terms to query catalog for CanESM5 data
# activity_id: Selecting CMIP for historical and ScenarioMIP for future projections
activity_ids = ['ScenarioMIP', 'CMIP'] 

# source_id: Models selected earlier
source_id = ['CanESM5' ,'CESM2']

# experiment_id: I chose the historical data and the ssp370 projection as my two time experimental configurations
experiment_ids = ['historical', 'ssp370']

# member_id: Changed the ensemble member here because there was more data available
member_id = 'r10i1p1f1'

# table_id: Selecting monthly atmospheric data, which is the table that precipitation is stored in. 
table_id = 'Amon' 

# variable_id:  surface air temperature
variable_id = ['tas', 'pr'] 

In [4]:
# Search through catalog, store results in "res" variable
res = catalog.search(activity_id=activity_ids, source_id=source_id, 
                     experiment_id=experiment_ids, table_id=table_id, variable_id=variable_id,
                     member_id=member_id)

# Display data frame associated with results
display(res.df)

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,zstore,dcpp_init_year,version
0,CMIP,NCAR,CESM2,historical,r10i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20190313
1,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/CCCma/CanESM5/histor...,NaN,20190429
2,CMIP,CCCma,CanESM5,historical,r10i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/CMIP/CCCma/CanESM5/histor...,NaN,20190429
3,ScenarioMIP,CCCma,CanESM5,ssp370,r10i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429
4,ScenarioMIP,CCCma,CanESM5,ssp370,r10i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/CCCma/CanESM5...,NaN,20190429
5,CMIP,NCAR,CESM2,historical,r10i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/CMIP/NCAR/CESM2/historica...,NaN,20200124
6,ScenarioMIP,NCAR,CESM2,ssp370,r10i1p1f1,Amon,tas,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/NCAR/CESM2/ss...,NaN,20200528
7,ScenarioMIP,NCAR,CESM2,ssp370,r10i1p1f1,Amon,pr,gn,s3://cmip6-pds/CMIP6/ScenarioMIP/NCAR/CESM2/ss...,NaN,20200528


In [5]:
# Read in the historical data file for CESM
hist_tas_cesm = xr.open_zarr(res.df['zstore'][0], storage_options={'anon': True})
hist_pr_cesm = xr.open_zarr(res.df['zstore'][5], storage_options={'anon': True})

# Read in 370 projections for CESM
tas_cesm_370 = xr.open_zarr(res.df['zstore'][6], storage_options={'anon': True})
pr_cesm_370 = xr.open_zarr(res.df['zstore'][7], storage_options={'anon': True})

# Read in the historical data file for GFDL
hist_tas_can = xr.open_zarr(res.df['zstore'][2], storage_options={'anon': True})
hist_pr_can = xr.open_zarr(res.df['zstore'][1], storage_options={'anon': True})

# Read in 370 projections for CAN
tas_can_370 = xr.open_zarr(res.df['zstore'][3], storage_options={'anon': True})
pr_can_370 = xr.open_zarr(res.df['zstore'][4], storage_options={'anon': True})

In [8]:
# Concatenate historical and future projections for CESM
tas_cesm = xr.concat([hist_tas_cesm, tas_cesm_370], dim="time")
pr_cesm = xr.concat([hist_pr_cesm, pr_cesm_370], dim="time")

# Concatenate historical and future projections for CAN
tas_can = xr.concat([hist_tas_can, tas_can_370], dim="time")
pr_can = xr.concat([hist_pr_can, pr_can_370], dim="time")


## Regional Averages

In [9]:
tas_cesm

<xarray.Dataset>
Dimensions:    (lat: 192, nbnd: 2, lon: 288, time: 3012)
Coordinates:
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
    lat_bnds   (lat, nbnd) float64 -90.0 -89.53 -89.53 ... 89.53 89.53 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 355.0 356.2 357.5 358.8
    lon_bnds   (lon, nbnd) float64 -0.625 0.625 0.625 ... 358.1 358.1 359.4
  * time       (time) object 1850-01-15 12:00:00 ... 2100-12-15 12:00:00
    time_bnds  (time, nbnd) object dask.array<chunksize=(1980, 2), meta=np.ndarray>
Dimensions without coordinates: nbnd
Data variables:
    tas        (time, lat, lon) float32 dask.array<chunksize=(600, 192, 288), meta=np.ndarray>
Attributes: (12/49)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   674885.0
    branch_time_in_parent:  306600.0
    case_id:                24
    ...                     ...
    variable_id:            tas
    variant_info:           CMIP6 20th century experiments (1850-2014) with C...
    variant_label:          r10i1p1f1
    status:                 2019-10-25;created;by nhn2@columbia.edu
    netcdf_tracking_ids:    hdl:21.14100/e47b79db-3925-45a7-9c0a-6799c2f1e8ae...
    version_id:             v20190313

In [10]:
# Extract variables to store as xarray
tas_cesm_xr = tas_cesm['tas']
pr_cesm_xr = pr_cesm['pr']

tas_can_xr = tas_can['tas']
pr_can_xr = pr_can['pr']

# Define region of interest
lat_min, lat_max = -11, 28
lon_min, lon_max = 92, 141

In [37]:
# Define function to generate area weights
def weights(dat):
    # Calculate weighting factor = cosine of latitude
    coslat = np.cos(np.deg2rad(dat.lat))
    weight_factor = coslat / coslat.mean(dim='lat')
    
    # Weight all points by the weighting factor
    computed_weight = dat * weight_factor
    
    # Return the set of weights: this has dimension equal to that of the input data
    return computed_weight

# Weight our INM precipitation data
tas_cesm_xr = weights(tas_cesm_xr)
pr_cesm_xr = weights(pr_cesm_xr)
tas_can_xr = weights(tas_can_xr)
pr_can_xr = weights(pr_can_xr)

In [13]:
arrays = [tas_cesm_xr, pr_cesm_xr, tas_can_xr, pr_can_xr]
names = ["tas_cesm", "pr_cesm", "tas_can", "pr_can"]

clean_vars = {}

for array, name in zip(arrays, names):
    # Define logical mask: True when lat/lon inside the valid ranges, False elsewhere
    lat_array = (array.lat >= lat_min) & (array.lat <= lat_max)
    lon_array = (array.lon >= lon_min) & (array.lon <= lon_max)

    # Find points where the mask value is True, drop all other points
    clean_array = array.where(lat_array & lon_array, drop=True)

    # Average over lat, lon dimensions to get a time series
    clean_vars[f"clean_{name}"] = clean_array.mean(dim=["lat", "lon"])


In [20]:
tas_cesm = clean_vars['clean_tas_cesm']
pr_cesm = clean_vars['clean_pr_cesm']

tas_can = clean_vars['clean_tas_can']
pr_can = clean_vars['clean_pr_can']

In [21]:
dfs = [tas_cesm, pr_cesm, tas_can, pr_can]

for df in dfs:
    # Converts calendar to standard format to troublshoot slicing errors
    df = df.convert_calendar('standard', use_cftime=False)

    # Select a time period of interest
    df = df.sel(time=slice('1850-01-01', '2099-12-31'))

In [24]:
dfs = [tas_cesm, pr_cesm, tas_can, pr_can]
names = ["tas_cesm", "pr_cesm", "tas_can", "pr_can"]

clean_trends = {}

for df, name in zip(dfs, names):
    # Calculate the annual mean precipitation
    mean_df = df.groupby('time.year').mean()

    # Calculate best-fit parameters for the linear polynomial fit of precipitation to year
    poly_df = np.polyfit(mean_df.year, mean_df, 1)

    # Generate a polynomial object using those best-fit parameters
    clean_trends[f"trend_{name}"] = np.poly1d(poly_df)


In [35]:
# Define min/max bounds for Southeast Asia
region = [92, -11, 141, 28] 

# Time periods of interest
per_early = [1975, 2025]
per_late = [2026, 2075]

# Fix datetime format
tas_cesm['time'] = tas_cesm.time.astype('datetime64[ns]')
pr_cesm['time'] = pr_cesm.time.astype('datetime64[ns]')
tas_can['time'] = tas_can.time.astype('datetime64[ns]')
pr_can['time'] = pr_can.time.astype('datetime64[ns]')

# Sort by time for filtering later on
tas_cesm = tas_cesm.sortby('time')
pr_cesm = pr_cesm.sortby('time')
tas_can = tas_can.sortby('time')
pr_can = pr_can.sortby('time')

In [ ]:
# Slice the data to the time periods
# INM historical
inm_hist = inm.sel(time=slice(str(per_early[0])+"-01-01", str(per_early[1])+"-12-31"))
# INM future
inm_fut = inm.sel(time=slice(str(per_late[0])+"-01-01", str(per_late[1])+"-12-31"))
# GFDL historical
gfdl_hist = gfdl.sel(time=slice(str(per_early[0])+"-01-01", str(per_early[1])+"-12-31"))
# GFDL future
gfdl_fut = gfdl.sel(time=slice(str(per_late[0])+"-01-01", str(per_late[1])+"-12-31"))